In [1]:
# Library imports
import numpy as np
from scipy.stats import norm

In [2]:
# Load datasets
train_data = np.load("./Assignment1-Dataset/train_data.npy")
train_label = np.load("./Assignment1-Dataset/train_label.npy")
test_data = np.load("./Assignment1-Dataset/test_data.npy")
test_label = np.load("./Assignment1-Dataset/test_label.npy")

In [3]:
# Sanity checks
print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

(50000, 128)
(50000, 1)
(10000, 128)
(10000, 1)


In [4]:
def generate_gaussian_weights(num_neurons, num_features):
    '''
    Generate weights taken from the Gaussian distribution, and based on the number of neurons within the hidden layer
    as well as the number of features of the dataset
    
    Output is weights matrix of size (num_neurons, num_features)
    '''
    
    weights = norm.rvs(size = [num_neurons, num_features], random_state=1)
    
    return weights

def generate_gaussian_bias(num_neurons):
    '''
    Generate bias vector from the Gaussian distribution, and based on the number of neurons within the hidden layer
    
    Output is bias vector of size (num_neurons)
    '''
    
    bias = norm.rvs(size = num_neurons, random_state=1)
    
    return bias


In [5]:
def calc_z(data_vector, weights_matrix, bias_vector):
    '''
    Calculate the z value for all the neurons within the specific hidden layer, obtained by taking the dot product
    between the weights matrix and data vector.  The bias vector is then added onto the product of the two.
    
    The output vector then represents the input value to be used for the activation function of all the neurons within
    the specific hidden layer.
    '''
    
    return weights_matrix.dot(data_vector) + bias_vector

In [6]:
def run_activation_func(activation_func, z):
    '''
    Calculates the value after the z has been computed and puts it inside the non-linear activation function
    that we have for that hidden layer
    '''
    
    if activation_func == 'relu':
        return np.maximum(0, z)
    
    if activation_func == 'softmax':
        return np.divide(np.exp(z), np.sum(np.exp(z)))

In [7]:
def encode_label_vector(label_vector):
    '''
    Encode the label vector of our dataset, such that we can use it for the computation of the MSE.
    This is because the labels are labelled as integers 0 to 9, whereas for MSE to work, we need to
    create a array vector of size 10 for every single observation, where the value 1 is set on the index of the correct observation
    '''
    
    num_classes = np.unique(train_label).size
    
    encoded_label_vector = []
    
    for label in label_vector:
        encoded_label = np.zeros(num_classes)
        encoded_label[label] = 1
        encoded_label_vector.append(encoded_label)
    
    encoded_label_vector = np.array(encoded_label_vector)
    
    return encoded_label_vector
    

def calculate_MSE(pred, actual):
    '''
    Calculates the Mean Squared Error between the prediction of the NN and actual class
    
    Note:
    This works because the pred value is the softmax of the output of the NN and
    the actual is adjusted such that the values are between 0 and 1
    '''
    error = np.subtract(pred, actual)
    squared_error = np.square(error)
    return np.sum(squared_error)

In [8]:
def calc_delta_softmax(layer_output, encoded_label_vector):
    '''
    Calculates the delta value for the final/output layer, which will be used for backpropagation
    
    Note:
    This function expects to receive the output of the softmax activation function within the output layer
    and also the encoded label vector which would have values 0 and 1 exclusively
    
    delta = activation_output - y
    
    Output: vector of size num of classes to be predicted
    '''
    
    return np.subtract(layer_output, encoded_label_vector)


def calc_delta_hidden(l_plus_one_delta, l_plus_one_weights, layer_output):
    '''
    Calculates the delta value for the hidden layers, which will be used for backpropagation
    
    This function is only to be used to calculate the delta values of the hidden layers.  Output layer delta should be
    the calc_delta_softmax, and there is no delta term needed to be calculated for input layer (NN layer 1)
    
    delta = (weights for next layer . delta for next layer) .* (activation_func output for this layer .* (1 - activation_func output for this layer))
    '''
    
    first_part = (l_plus_one_weights.T).dot(l_plus_one_delta)
    second_part = np.multiply(layer_output, np.ones(layer_output.size) - layer_output)
    return  np.multiply(first_part, second_part)

In [12]:
# Let's do a test-run of the functions above in setting up feedforward

# Make sure to encode the label vector
encoded_label_vector = encode_label_vector(train_label)

hidden_layers = 3 # this should be int
hidden_layers_activation_func = ['relu', 'relu', 'relu']
num_neurons = [5,3,6] # this should be a list containing int per hidden layer
num_classes = np.unique(train_label).size

weight_matrix = []
bias_vector = []

# initialise the weights
for layer_num, layer in enumerate(range(hidden_layers)):
    
    # If we are instantiating the details for the first hidden layer, then make the following adjustments
    # which would otherwise be not required for subsequent hidden layers
    if layer_num == 0:
        # The input features would be the shape of our dataset instead of num of features from previous layer
        num_input_features = train_data.shape[1]
    else:
        num_input_features = num_neurons[layer_num - 1]
    
    # check how many neurons should be in this layer
    neuron_num = num_neurons[layer_num]
    
    layer_weights = generate_gaussian_weights(neuron_num, num_input_features)
    layer_bias = generate_gaussian_bias(neuron_num)
    
    weight_matrix.append(layer_weights)
    bias_vector.append(layer_bias)
    
    print(f'Weight and bias generated for hidden layer {layer_num + 1} with weight shape {weight_matrix[layer_num].shape} and bias shape of {bias_vector[layer_num].shape}')

   
# insantiate the parts for the output layer
# need to be very careful with the use of -1 indices, in the event that we incorporate output layer to our hidden layer variables
weight_matrix.append(generate_gaussian_weights(num_classes, num_neurons[-1]))
bias_vector.append(generate_gaussian_bias(num_classes))


# feedforward part
layer_z = []
layer_output = []
for layer_num, layer in enumerate(range(hidden_layers)):
    
    if layer_num == 0:
        input_data = train_data[0] # hard coding this for now, will set batches later
    else:
        # extract the output of the previous layer
        input_data = layer_output[layer_num - 1]
        
    z = calc_z(input_data, weight_matrix[layer_num], bias_vector[layer_num])
    a = run_activation_func(hidden_layers_activation_func[layer_num], z)
    
    layer_output.append(a)
    
    
# Calculation for the output layer
z = calc_z(layer_output[-1], weight_matrix[-1], bias_vector[-1])
a = run_activation_func('softmax', z)
layer_z.append(z) # to be used for backpropagation
layer_output.append(a)

# Calculate the error
print(calculate_MSE(layer_output[-1], encoded_label_vector[0]))


# Perform the backpropagation
# instantiate the delta list for hidden_layers + output layer
delta = [0 for i in range(hidden_layers + 1)]

# for the output layer
delta[hidden_layers] = (calc_delta_softmax(layer_output[-1], encoded_label_vector[0])) # note that the zero is hardcoded for example 0

for layer_num, layer in reversed(list(enumerate(range(hidden_layers)))):
    # basically prepending to the start of the list, in order to retain layer order in delta list/matrix
    print(f'Layer num is {layer_num}')
    delta[layer_num] = calc_delta_hidden(delta[layer_num + 1], weight_matrix[layer_num + 1], layer_output[layer_num])

Weight and bias generated for hidden layer 1 with weight shape (5, 128) and bias shape of (5,)
Weight and bias generated for hidden layer 2 with weight shape (3, 5) and bias shape of (3,)
Weight and bias generated for hidden layer 3 with weight shape (6, 3) and bias shape of (6,)
1.9998233671553345
Layer num is 2
Layer num is 1
Layer num is 0
